In [1]:
import pandas as pd
import numpy as np


####solver
from scipy.optimize import fsolve


import matplotlib.pyplot as plt
%matplotlib inline


## Short Interest Tree Model

### Example from book "Fixed Income Securities by Pietro Veronesi

I am using this book as a reference to create my functions then to test them. 
At a later stage, I will try to calibrate "theta".

### Ho-Lee and Black-Derman-Toy model.

In [2]:
def c2c(r, n):
    """
    Continuous to compounded rate

    Args:
        r (float): continuous interest rate
        n (int): number of compounding

    Returns:
        float: compounded interest rate
    """
    return n * (np.exp(r/n)-1)

In [42]:
class Tree(object):
    
    '''
    Class to calibrate Binamial Tree to fit the interest rate
    using Ho Lee and Black Derman Toy implementation
    ============================
    n: number of time period
    T: number of years. 
    dt: T/n
    zcb: array, price of zero coupon bonds. 
        Make sure these are ZCB. Only check is if zcb >1, then devide by 100. 
        
    sigma = Annualised Volatility (standard deviation)
    
    =============================
    Assumption: dt is constant
    Future development ideas: 
        If dt between each price not constant, 
        implement raw interpolation  
    
    =============================
 
    Ho Lee:
    Black Derman Toy: 
    
    =============================
    '''
    
    def __init__(self, zcb, T,  sigma ):
        
        self.zcb = np.array(zcb) # Array
        self.n = len(self.zcb) # idea 
        self.T = T
        self.sigma = sigma # Annualised volatility 
        self.dt = T/self.n
        self.time = np.arange(self.dt, self.T +self.dt, self.dt) # easier to time
        self.rates = np.zeros((self.n, self.n))
        self.thetas = np.nan # store theta's value once calibrated
        

        
        # if ZCB > 1 ==> ZCB quoted per $100. 
        if self.zcb[-1]>1.0:
            self.zcb /= 100
        
        # Extract first interest rate (Trivial)
        self.rates[0,0] = -np.log(self.zcb[0])/self.dt
            



        
    @staticmethod
    def forward_tree(r0,sigma,dt,thetas):
        """
        Forward tree valuation
        Works for both Ho Lee and BDT model (change r_0 to ln r_0)

        Args:
            r0 (float): _description_
            sigma (float): _description_
            dt (float): _description_
            thetas (array / list): _description_

        Returns:
             nxn array: interest rates
             
        =========================
        Uses backward tree for Bond evaluation
        Future area of improvement, maybe use
        backward_tree method
        """
        n = len(thetas)
        tree_rate = np.zeros((n+1,n+1))
        tree_rate[0,0] = r0
        tree_zcb = np.zeros((n+2, n+2))
        tree_zcb[:, -1] = 1.0 # Could be 100.0

        for i in range(n):
            tree_rate[0,i+1] = tree_rate[0,i] + thetas[i] * dt \
                + sigma * np.sqrt(dt)
            
            
            # Vectorise calculation by -2 sigma on each row
            # (column-wise operation)
            
            # ignore first row (already calculated)
            tree_rate[1:i+2,i+1] = tree_rate[0,i+1] \
                - 2 * np.arange(1,i+2) * sigma * np.sqrt(dt)
            
        # Calculate ZCB backward
        

        for i in np.arange(n, -1, -1):

            
            tree_zcb[0:i+1,i] = np.exp(-tree_rate[:i+1,i] * dt) \
                * 0.5 * (tree_zcb[0:i+1,i+1] + tree_zcb[1:i+2,i+1])
            

        return tree_rate, tree_zcb
            
    def _fit_theta(self):

        thetas=[]
        P=self.zcb
        r0 = self.rates[0,0]

        for i in P[1:]:
            p0=i
            func=(lambda t: self.forward_tree(r0,self.sigma,self.dt,thetas+[t])[1][0,0]-p0)
            new_theta=fsolve(func,0.002)
            thetas.append(new_theta[0])
            
        self.thetas = thetas


    def ho_lee(self):
        
        '''
        
        In financial mathematics, the Ho–Lee model is a short rate model widely used 
        in the pricing of bond options, swaptions and other interest rate derivatives,
        and in modeling future interest rates. It was developed in 1986 by Thomas Ho and Sang Bin Lee.
        (from Wikepedia: https://en.wikipedia.org/wiki/Ho%E2%80%93Lee_model)
        
        - Risk Neutral probability = 0.5
        
        Drawback: Ho-Lee model allows negative interest rate

        '''
        
        
        self._fit_theta()
        temp = self.forward_tree(self.rates[0,0], self.sigma, self.dt, self.thetas)
        self.rates = temp[0]

        return self.thetas

    # TODO: Work in Progress
    def bdt(self):
        '''
        In mathematical finance, the Black–Derman–Toy model (BDT) is a popular short rate model
        used in the pricing of bond options, swaptions and other interest rate derivatives.
        Wikipedia: https://en.wikipedia.org/wiki/Black%E2%80%93Derman%E2%80%93Toy_model
        
        - Risk Neutral probability = 0.5
        
        '''
        
        self.rates[0,0] = np.log(self.rates[0,0])
        
        self._fit_theta()
        temp = self.forward_tree(self.rates[0,0], self.sigma, self.dt, self.thetas)
        self.rates = np.exp(temp[0])
   
        # z_i = ln(r_i) <=> r_i = exp(z_i)
       
        
        return self.thetas


    # Future Development
    @staticmethod
    def backward_tree(rates, dt, last = 100):
        '''
        backward tree
        rates: Binomial Tree of interest rate
                used to price bond.
        last: value at maturity

        '''
        n = len(rates)
        tree = np.zeros((n + 1, n + 1)) # bond price tree
        tree[:,-1] = last

        for i in range(n-1,-1,-1):
            for j in range(n-1,-1,-1):
                if j<=i:
                    # if p != 0.5 (as per model), change this line
                    tree[j,i] = np.exp(-rates[j,i] * dt) \
                        * 0.5 * (tree[j,i+1] + tree[j+1, i+1])


        return tree
     

## Testing Class

In [40]:
hl2=Tree(pzcb, 5.5, 0.0173)


hl2.ho_lee()


[0.015677949304024934,
 0.02182235310344373,
 0.014374983432949849,
 0.017319055014414046,
 0.007878767580903848,
 0.00042109515425436825,
 -46.03090405617534,
 92.06487308349355,
 -46.02100335324746,
 0.0012003362863116914]

In [43]:
hl2=Tree(pzcb, 5.5, 0.0173)


hl2.bdt()


[16.290538785100303,
 -8.115608064794046,
 0.01437498343202353,
 0.01731905501478457,
 0.007878767580901967,
 0.0004210951542564152,
 -46.03090405617534,
 92.06487308349355,
 -46.02100335324746,
 0.0012003362863106978]

In [46]:
pd.DataFrame(hl2.forward_tree(0.0174, 0.0173, 0.5, hl2.bdt())[1])

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.480188e-10,1.401909e-10,7.894277e-09,5.918542e-08,4.509333e-07,0.000003,0.000027,0.000219,1.770763e-08,0.014383,0.119178,1.0
1,0.000000e+00,1.584334e-10,8.813059e-09,6.527041e-08,4.912485e-07,0.000004,0.000029,0.000230,1.836955e-08,0.014739,0.120645,1.0
2,0.000000e+00,0.000000e+00,9.838775e-09,7.198102e-08,5.351680e-07,0.000004,0.000031,0.000241,1.905621e-08,0.015104,0.122130,1.0
3,0.000000e+00,0.000000e+00,0.000000e+00,7.938155e-08,5.830141e-07,0.000004,0.000033,0.000253,1.976854e-08,0.015478,0.123633,1.0
4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.351378e-07,0.000005,0.000035,0.000266,2.050750e-08,0.015862,0.125155,1.0
5,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000005,0.000037,0.000279,2.127408e-08,0.016254,0.126695,1.0
6,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000040,0.000293,2.206932e-08,0.016657,0.128254,1.0
7,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000308,2.289428e-08,0.017070,0.129833,1.0
8,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,2.375008e-08,0.017492,0.131431,1.0
9,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.017926,0.133049,1.0


In [23]:
a = hl2.forward_tree(0.0174, 0.0173, 0.5, theta_0)[0]

In [24]:
b = hl2.forward_tree(0.0174, 0.0173, 0.5, theta)[0]

In [25]:
a[:, -1] - b[:,-1]

array([6.06977816e-07, 6.06977816e-07, 6.06977816e-07, 6.06977816e-07,
       6.06977816e-07, 6.06977816e-07, 6.06977816e-07, 6.06977816e-07,
       6.06977816e-07, 6.06977816e-07, 6.06977816e-07])

In [309]:
theta

[0.015674,
 0.021824,
 0.014374,
 0.017324,
 0.007873,
 0.000423,
 -0.000628,
 0.004322,
 0.009271,
 0.001202]

In [33]:
hll = Tree2()

In [34]:
hll.fit(pzcb, 0.0173, 0.5)

c:\Users\jules\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [35]:
hll.thetas

[0.015677949304024857,
 0.02182235310344355,
 0.014374983432950177,
 0.01731905501441534,
 0.00787876758090165,
 0.0004210951542544909,
 -46.03090405617909,
 92.06487308350104,
 -46.02100335325977,
 0.0012003363034583142]

In [36]:
theta_0

[0.015677949304024403,
 0.021822353103444363,
 0.014374983432949618,
 0.017319055014415174,
 0.007878767580901651,
 0.00042109515425705317,
 -46.03090405617908,
 92.06487308350104,
 -46.02100335325978,
 0.0012003363034622143]

In [74]:
temp = pd.DataFrame(hl2.forward_tree(0.0174,0.0173,0.5,theta))

In [42]:
hl2.thetas = theta

In [12]:
pzcb= price
pzcb=[item/100 for item in pzcb]


In [103]:
price

[99.1338,
 97.8925,
 96.1462,
 94.1011,
 91.7136,
 89.2258,
 86.8142,
 8405016,
 82.1848,
 79.7718,
 77.4339]

In [30]:
hl2.thetas

[0.015677949304024857,
 0.02182235310344355,
 0.014374983432950177,
 0.01731905501441534,
 0.00787876758090165,
 0.0004210951542544909,
 -46.03090405617909,
 92.06487308350104,
 -46.02100335325977,
 0.0012003363034583142]

In [100]:
hl2.rates_tree

,0,1,2,3,4,5,6,7,8,9,10
0,0.017399,0.037471,0.060616,0.080036,0.100928,0.117101,0.129544,-22.873675,23.170995,0.172726,0.185559
1,0.000000,0.013005,0.036150,0.055570,0.076463,0.092635,0.105078,-22.898141,23.146529,0.148260,0.161093
2,0.000000,0.000000,0.011684,0.031104,0.051997,0.068169,0.080612,-22.922607,23.122063,0.123794,0.136627
3,0.000000,0.000000,0.000000,0.006638,0.027531,0.043703,0.056147,-22.947073,23.097597,0.099328,0.112161
4,0.000000,0.000000,0.000000,0.000000,0.003065,0.019237,0.031681,-22.971538,23.073131,0.074862,0.087695
5,0.000000,0.000000,0.000000,0.000000,0.000000,-0.005229,0.007215,-22.996004,23.048665,0.050396,0.063230
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.017251,-23.020470,23.024199,0.025931,0.038764
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-23.044936,22.999733,0.001465,0.014298
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.975268,-0.023001,-0.010168
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.047467,-0.034634


In [83]:
t = Tree(price, 5.5, 0.0173)
pd.DataFrame(t.fit_theta())

AttributeError: 'Tree' object has no attribute 'myholee'

In [70]:
t.zcb

array([9.913380e-01, 9.789250e-01, 9.614620e-01, 9.410110e-01,
       9.171360e-01, 8.922580e-01, 8.681420e-01, 8.405016e+04,
       8.218480e-01, 7.977180e-01, 7.743390e-01])

In [10]:
# Table 10.11 Zero Coupon Bond Prices on January 8, 2002,
# Source The Wall Street Journal
# Taken from Fixed Income Securities - Pietro Veronesi || p.370


maturity = np.arange(0.5,6.0,0.5)

price = [99.1338, 97.8925, 96.1462,
         94.1011, 91.7136, 89.2258,
         86.8142, 8405016, 82.1848,
         79.7718, 77.4339]

yield_ = [1.74, 2.13, 2.62, 3.04,
          3.46,3.8,4.04,4.21,4.36, 
          4.52,4.65]

df = pd.DataFrame([maturity, price, yield_]).T
df.columns = ["Maturity", "Price", "Yield"]

In [41]:
np.append([0,1,2,3],[2])

array([0, 1, 2, 3, 2])

##### Table: 11.2; p 384

In [4]:
t = Tree(1.5,3)
t.v = t.ho_lee(0.0174,0.0173, theta)
t.tree_bond(t.v)


NameError: name 'theta' is not defined

In [ ]:
pd.DataFrame(Tree(1.5,3).tree_bond(t[:3,:3]))

96.14627330741644

In [15]:
theta = [0.015674,0.021824,0.014374,0.017324,0.007873,0.000423,-0.000628,0.004322,0.009271,0.001202]
pd.DataFrame(Tree(5,10).ho_lee(0.0174,0.0173, theta))

TypeError: Tree.__init__() missing 1 required positional argument: 'sigma'

##### Black Derman & Toy
Table 11.3; p385

In [ ]:
theta1 = [0.7182,0.6916,0.3348,0.3379,0.1182,-0.023,-0.0438,0.0455,0.1281,-0.0126]
pd.DataFrame(Tree(5,10).bdt(0.0174,0.2142,theta1))

,0,1,2,3,4,5,6,7,8,9,10
0,0.0174,0.028992,0.047670,0.065573,0.090339,0.111512,0.128264,0.146007,0.173794,0.215590,0.249272
1,0.0000,0.021415,0.035211,0.048435,0.066729,0.082369,0.094743,0.107849,0.128373,0.159247,0.184126
2,0.0000,0.000000,0.026009,0.035777,0.049290,0.060842,0.069982,0.079663,0.094824,0.117629,0.136005
3,0.0000,0.000000,0.000000,0.026427,0.036408,0.044941,0.051693,0.058844,0.070042,0.086887,0.100461
4,0.0000,0.000000,0.000000,0.000000,0.026893,0.033196,0.038183,0.043465,0.051737,0.064179,0.074206
5,0.0000,0.000000,0.000000,0.000000,0.000000,0.024520,0.028204,0.032106,0.038216,0.047406,0.054813
6,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.023715,0.028228,0.035017,0.040488
7,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017517,0.020851,0.025865,0.029906
8,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015402,0.019106,0.022090
9,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014112,0.016317


### Caps and Floor

Caps are like european calls <br>
Floors are like european puts

In [ ]:
class Option_IR(object):
    
    def __init__(self,T, n):
        ''' in case I need to write something'''
        
        self.T = T        
        self.n = n

        self.dt = T/n
       
    
    def ctns_rate(self,r):
        
        return (np.exp(r*self.dt)-1)/self.dt
        
        
        
    def european(self, tree_rate, Strike,N, cp = "cap"):



        p = 0.5
        
        tree_ctns = self.ctns_rate(tree_rate)
        
        
        tree_cf = np.zeros((self.n+1,self.n+1)) # create an empty array for Cash-Flow tree
        
        for i in range(self.n+1):
            for j in range(self.n+1):
                if j>i:
                    tree_cf[j,i] = 0
                else:
                    if cp =='cap':
                        tree_cf[j,i] = self.dt * N * max(tree_ctns[j,i] - Strike,0)
                    else:
                        tree_cf[j,i] = self.dt * N * max(Strike - tree_ctns[j,i],0)
                    
        tree_eu = np.zeros((self.n+1,self.n+1))
        
        
        #intresic value
        for j in range(self.n+1):
            tree_eu[j,-1] = np.exp(-tree_rate[j,-1] * self.dt) * tree_cf[j,-1]
            
            
                

        #Backward tree (European option)
        for i in range(self.n,0,-1):
            for j in range(i):
                tree_eu[j,i-1] = np.exp(-tree_rate[j,i-1] * self.dt)*(( p * tree_eu[j,i] + (1-p) * tree_eu[j+1,i])+tree_cf[j,i-1])

        return tree_cf, tree_eu
    

    
    def swap_rate(self, zcb):
        
        return 1/self.dt * (1 - zcb[-1])/ (sum(zcb))
    

    def swap(self, tree_rate, c, N):
        
        p = 0.5
        
        tree_ctns = self.ctns_rate(tree_rate)
        
        
        tree_cf = np.zeros((self.n+1,self.n+1)) # create an empty array for Cash-Flow tree
        
        for i in range(self.n+1):
            for j in range(self.n+1):
                if j>i:
                    tree_cf[j,i] = 0
                else:
                    tree_cf[j,i] = self.dt * N * (tree_ctns[j,i] - c)
                    
        tree_eu = np.zeros((self.n+1,self.n+1))
        
        
        #intresic value
        for j in range(self.n+1):
            tree_eu[j,-1] = np.exp(-tree_rate[j,-1] * self.dt) * tree_cf[j,-1]
            
            
                

        #Backward tree (European option)
        for i in range(self.n,0,-1):
            for j in range(i):
                tree_eu[j,i-1] = np.exp(-tree_rate[j,i-1] * self.dt)*(( p * tree_eu[j,i] + (1-p) * tree_eu[j+1,i])+tree_cf[j,i-1])

        return tree_cf, tree_eu


In [ ]:
pd.DataFrame(Option_IR(4.5,9).swap(y,c, 100)[1])

,0,1,2,3,4,5,6,7,8,9
0,0.00689,4.272425,8.188949,11.385667,13.866054,15.227770,15.502382,14.717643,12.586277,8.204988
1,0.00000,-1.519939,2.046496,5.049234,7.484485,9.030847,9.717720,9.580177,8.417565,5.582189
2,0.00000,0.000000,-2.785965,0.050314,2.445034,4.146658,5.180691,5.581978,5.209567,3.596844
3,0.00000,0.000000,0.000000,-3.826893,-1.466314,0.361367,1.677267,2.512431,2.767122,2.103602
4,0.00000,0.000000,0.000000,0.000000,-4.463328,-2.535859,-0.996939,0.179442,0.922178,0.985778
5,0.00000,0.000000,0.000000,0.000000,0.000000,-4.732939,-3.020801,-1.580577,-0.463315,0.151905
6,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.542809,-2.901052,-1.499293,-0.468546
7,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.887730,-2.271460,-0.929320
8,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.845639,-1.271028
9,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.524176


##### Assign your tree to a variable x:

In [ ]:
x = Tree(1,2).bdt(0.0174,0.2142,theta1)
pd.DataFrame(x)

,0,1,2
0,0.0174,0.028992,0.047670
1,0.0000,0.021415,0.035211
2,0.0000,0.000000,0.026009


##### Cash Flow tree: (p.390)

In [ ]:
pd.DataFrame(Option_IR(1,2).european(x,0.025,100,'cap')[0]) # Cash flow( Please note it arrives a T+1)

,0,1,2
0,0.0,0.210176,1.162114
1,0.0,0.000000,0.526160
2,0.0,0.000000,0.058947


##### Cap Value Tree: (p.391)

In [ ]:
pd.DataFrame(Option_IR(1,2).european(x,0.025,100,'cap')[1])

,0,1,2
0,0.647167,1.021126,1.134743
1,0.000000,0.284518,0.516978
2,0.000000,0.000000,0.058185


##### Cash Flow, 5-year Cap: Panel A, p 392

In [ ]:
y = Tree(4.5,9).bdt(0.0174,0.2142,theta1)
pd.DataFrame(Option_IR(4.5,9).european(y,0.025,100,'cap')[0])

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.210176,1.162114,2.082966,3.370518,4.483962,5.373335,6.323442,7.828410,10.131965
1,0.0,0.000000,0.526160,1.201337,2.142751,2.954425,3.601152,4.290489,5.379150,7.037926
2,0.0,0.000000,0.058947,0.554951,1.245117,1.838851,2.311063,2.813549,3.605376,4.807823
3,0.0,0.000000,0.000000,0.080115,0.587084,1.022503,1.368336,1.735888,2.314145,3.190092
4,0.0,0.000000,0.000000,0.000000,0.103738,0.423658,0.677501,0.947041,1.370591,2.011013
5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.170201,0.368239,0.679156,1.148637
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.171417,0.516267
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.051672
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


##### Cash Flow, 5-year Cap: Panel B, p 392

In [ ]:
pd.DataFrame(Option_IR(4.5,9).european(y,0.025,100,'cap')[1])

,0,1,2,3,4,5,6,7,8,9
0,9.439097,12.18811,15.098181,17.348980,18.926699,19.437904,18.889858,17.286189,14.325789,9.096594
1,0.000000,6.85504,9.213622,11.251525,12.761801,13.423482,13.247149,12.249134,10.218022,6.499271
2,0.000000,0.00000,4.644049,6.450698,7.890201,8.680570,8.819725,8.328137,7.056592,4.533210
3,0.000000,0.00000,0.000000,2.841083,4.134157,5.003337,5.399971,5.317338,4.649420,3.054471
4,0.000000,0.00000,0.000000,0.000000,1.463359,2.242706,2.789097,3.028711,2.831020,1.947505
5,0.000000,0.00000,0.000000,0.000000,0.000000,0.516157,0.924069,1.302001,1.465404,1.121731
6,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.120979,0.231983,0.444260,0.507306
7,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012510,0.025240,0.051008
8,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


###### Zero Coupon Bond Price on January 8, 2002 (p 370)

In [ ]:
zcb_08 = [99.1338, 97.8925,
      96.462, 94.1011,
      91.7136, 89.2258,
      86.8142, 84.5016,
      82.1848, 79.7718, 77.4339]

In [ ]:
del zcb_08[-1] # remove the last element of the list
zcb = [i/100 for i in zcb_08] # divide each element by 100
zcb


[0.9913379999999999,
 0.9789249999999999,
 0.96462,
 0.941011,
 0.917136,
 0.8922580000000001,
 0.868142,
 0.845016,
 0.8218479999999999,
 0.797718]

##### Calculating c as per example 11.5 page 393. 
please note that the formula should be 2x (and not 1/2!)

In [ ]:
c = Option_IR(1,2).swap_rate(zcb)
c

0.04486177219546836